# Preprocess Data 
create .npy stacks of train/test/val data from raster data (from GEE) to use to train model(s)

## Import Packages

In [ ]:
# Import required packages
from datetime import datetime,date
import numpy as np
import calendar
from dateutil import parser
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import rasterio as rio
from rasterio.warp import reproject, Resampling
# from shapely.geometry import box
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.layers import Dropout, Dense, LSTM, RepeatVector, TimeDistributed, Input, Reshape, BatchNormalization
# from tensorflow.keras.models import Sequential, Model

# from keras.callbacks import EarlyStopping
# import pickle

# import tensorflow as tf #version 2.11.0
# import keras #v 2.11.0
import os

# import rasterio
from rasterio.enums import Resampling
# from rasterio.warp import calculate_default_transform#, reproject

# import json

from math import sqrt
# from sklearn.metrics import mean_squared_error
# from keras.callbacks import LearningRateScheduler


      
np.random.seed(5)



## Define Functions to Read in Raster and Point Data

In [ ]:

def turn_raster_output_to_gdf(dynamic_stack_fp, points, band_names_stack, prefire_months=120):
    
    """
    Extracts raster values at given points within a bounding box.

     Args:
    [str] dynamic_Stack_fp = Path to the raster file (gtiff file.
    [GeoDataFrame] points = point shapefile /Points to extract raster values from
    [list of str] band_names_stack = names of attributes to add to output gdf
    [int] prefire months = number of months before the fire occurred that we are interested in/ 
                            Number of prefire months for time series matching.

     Creates:
     [list] values (list): List of extracted values and corresponding points.
     
     
     Then..Turns output from the extract_points function into a geodataframe

     Returns: 
     [geodataframe] Extracted values from raster and geometry points in dataframe

    """

    raster = rio.open(dynamic_stack_fp)

    values = []

    point_num = len(points)#_clip)
    array = raster.read()

    # Loop through points, get the value of the raster at each xy
    for point in points['geometry']:
  
        x, y = point.xy[0][0], point.xy[1][0]
        row, col = raster.index(x, y)
        attrs = []
        band_nums = array.shape[0]

        for z in range(band_nums):
                attrs.append(array[z, row, col])
           
        values.append([attrs, point])
        
    # Close raster
    raster.close()
    
    #convert values to dataframe
    output_pd = pd.DataFrame(values)
    
    lis = output_pd[0].to_list() #list of attributes in format [precip1, temp1, ndvi1, precip2, temp2, ndvi2...]
    geom = output_pd[1].to_list() #geometry list (of point coords)
    # Turn output into labeled
    org = pd.DataFrame(lis, columns = band_names_stack)
    # read points into geodataframe
    return gpd.GeoDataFrame(org, geometry=geom, crs=points.crs)


def turn_raster_output_to_gdf_STATIC(dynamic_stack_fp, points, band_names_stack, prefire_months=120):
    
    """
    Extracts raster values at given points within a bounding box.
    For use with static rasters(e.g. slope (which doesnt change) instead of NDVI which is monthly)
    Difference from non-static version of function is that this passes if the point isn't located
    within a given raster tile bc these rasters are *multipart*

     Args:
    [str] dynamic_Stack_fp = Path to the raster file (gtiff file)
    [GeoDataFrame] points = point shapefile /Points to extract raster values from
    [str list] band_names_stack = names of attributes to add to output gdf
    [int] prefire months = number of months before the fire occurred that we are interested in/ 
                            Number of prefire months for time series matching.

     Creates:
     [list] values (list): List of extracted values and corresponding points.
     
     Then..Turns output from the extract_points function into a geodataframe

     Returns: 
     [geodataframe] Extracted values from raster and geometry points in dataframe

    """

    raster = rio.open(dynamic_stack_fp)
    
    #first ensure that raster and points are same crs:

    if raster.crs != points.crs:
        print("NEED TO REPROJECT FIRST! Cannot continue..")
        print(raster.crs,'is raster crs; points crs is:', points.crs)
        out_path = dynamic_stack_fp.replace('.tif','_'+str(points.crs).replace('epsg:','')+'.tif')
        reproject_raster(dynamic_stack_fp, points.crs, out_path)#output_raster_path='default')
#         raise
        raster = rio.open(out_path)
    
    values = []
    point_num = len(points)#_clip)
    array = raster.read()
    
    # Loop through points, get the value of the raster at each xy
    for point in points['geometry']:
        x, y = point.xy[0][0], point.xy[1][0]
#         print("x,y",x,y)
        row, col = raster.index(x, y)
#         print("r,c",row,col)
        attrs = []
        band_nums = array.shape[0]

        for z in range(band_nums):
            try:
                if np.isnan(array[z, row, col]) and z!=4: #when z=4, this is severity where there was an issue... all are nan (basically)
                    raise
                else:
#                 if not np.isnan(array[z, row, col]):
                #!= True and array[z, row, col] != 'nan' and array[z, row, col] is not None:\
#                     print(type(array[z,row,col]), array[z,row,col]) #np.float; e.g 9.0
                    attrs.append([array[z, row, col]] * prefire_months) #ERROR HERE
            
#                 else:
#                     print('val:',array[z, row, col])
#                     fake_app = -999999#[-999999 for i in range(prefire_months)]
#                     attrs.append(fake_app)
#                     values.append([attrs, point])
                    
            except:
                pass
        if len(attrs)>0:
            values.append([attrs, point]) ## in other function this is unindented to same as 'for z in range..'
            
    # Close raster
    raster.close()
    
        
    output_pd = pd.DataFrame(values)
    
    lis = output_pd[0].to_list() #list of attributes in format [precip1, temp1, ndvi1, precip2, temp2, ndvi2...]
    geom = output_pd[1].to_list() #geometry list (of point coords)
#     print(len(geom),'is len geom')
#     print("lis 0:",len(lis), len(lis[0]), len(lis[0][0]))
    # Turn output into labeled
    org = pd.DataFrame(lis, columns = band_names_stack)
    # read points into geodataframe
    return gpd.GeoDataFrame(org, geometry=geom, crs=points.crs)

def reproject_raster(input_raster_path, target_crs, output_raster_path='default'):
    """
    Reproject a raster file to a new coordinate reference system (CRS).

    Args:
        input_raster_path (str): The file path to the input raster file.
        target_crs (rasterio.crs.CRS): The target CRS to reproject the raster to.
        output_raster_path (str): The file path to the output raster file. Default is 'default'.
    
    Returns:
        None (saves the reprojected raster to the output_raster_path)

    """

    src = rio.open(input_raster_path)


    # Create an output raster file
    dst = rio.open(output_raster_path, 'w', driver='GTiff',
                        height=src.height, width=src.width,
                        count=src.count, dtype=src.dtypes[0],
                        crs=target_crs, transform=src.transform)

    # Loop through each band and perform the reprojection
    for band_idx in range(1, src.count + 1):
        reproject( #using rasterio.warp/
            
            source=rio.band(src, band_idx),
            destination=rio.band(dst, band_idx),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=dst.transform,
            dst_crs=dst.crs,
            dst_nodata=np.nan,
            resampling=Resampling.bilinear
        )
    # Close the files
    src.close()
    dst.close()


def convert_epoch_milli_to_datetime(milli):
  return(datetime.fromtimestamp(int(milli/1000)))

## Read In & Preprocess Data

### List filepaths to data on local machine (laptop) and set output folder

In [ ]:
# list filepaths to data

vcf_tree_fp = #tif of vcf pct tree per year
lai_fp = # tif for modis monthly LAI
geojson_fp = #points geojson (regularly sampled within fire boundary)
dynamic_stack_fp = #geotif of dynamic (precip, temp, ndvi)
dyn_stack_extracted_pts_fp = #shapefile of points
fire_bounds_fp = #fire bounds shapefile
static_fp = #data folder for static rasters
static_fp1 = #static rasters area tiled; reference part 1
static_fp2 = 
static_fp3 = 
static_fp4 = 
burn_main_dir = # directory with burn severity rasters for each fire

#name output file path where to save all data:
out_fp = #create new home dir 
    os.mkdir(out_fp)
os.chdir(out_fp)
os.getcwd()


### Read in Point geojson

In [ ]:
# Load in shapefile of points to extract from raster
points_shp = gpd.read_file(geojson_fp)


### Load in dynamic raster stack, rename bands, extract points from stack to create gdf

In [ ]:
# Read in time series raster stack

# Create band names list, created to match the order the bands come from google earth engine
band_names_stack = []

years = list(range(2000, 2022))
months = list(calendar.month_name[1:])
vars = ['temp','precip',  'NDVI']

for year in years:
  for month in months:
    for var in vars:
      band_names_stack.append(var + month[:3] + str(year))
      #note: NDVI data from Jan 2000 is a copy of Feb 2000 (as NDVI MODIS data doesnt exist for jan) but this month won't get used as there arent fires from jan 2000 so its ok

# Load in raster time series stack and extract points from the loaded in raster
# ras_id = '1_WtM8diGPc-swcu9xMFm0Bczqd4fFV1F' ##updated 7/19 '1J_BVg2o4rabBgXLJRJVKw1CvZO23hm7H'
# fdsafdsafdsa this raster is the old one with errors! switch to new one which is being generated in gee in cs17 on 7/11

time_series_gdf = turn_raster_output_to_gdf(dynamic_stack_fp, points_shp, band_names_stack, 120)
time_series_gdf.head()


### Divide all numeric columns by 100
purpose: to revert them back to original data values
(in gee I multiplied everythign by 100 so I could convert to int32 without losing precision)

In [ ]:
#remove ML Stack to free up ram
# os.remove('MLstack.tif')
print("make sure to check dtype of this array in case its in int and doesnt coorectly change to float")

#modify in place

# Select only the numeric columns (excluding the geometry column)
numeric_cols = time_series_gdf.select_dtypes(include=[pd.np.number]).columns

# Divide all numeric columns by 100
time_series_gdf[numeric_cols] = time_series_gdf[numeric_cols] / 100

time_series_gdf.head()
#NOTE: true min and max of MODIS NDVI 250m 16day product are: -2,000 and	10,000


In [ ]:
gdf_dtype = time_series_gdf[time_series_gdf.columns[1]].dtype
print(gdf_dtype, 'is datatype of geodataframe')
if gdf_dtype != 'float32' and gdf_dtype != 'float64':
  print("need to convert to float rather than int ")#"(though first check this datatype after dividing by 100 to seee if it converts automatically)")
  raise
time_series_gdf.head()


### Read in LAI Data (monthly)

In [ ]:
# lai_id = r'1Q4G3-ypzEfthwJzprHD1tT5irA2AXL0h'

# print(time_series_gdf)
lai_band_names = ['Band 1: Feb00', 'Band 2: Mar00', 'Band 3: Apr00', 'Band 4: May00', 'Band 5: Jun00', 'Band 6: Jul00', 'Band 7: Aug00', 'Band 8: Sep00', 'Band 9: Oct00', 'Band 10: Nov00', 'Band 11: Dec00', 'Band 12: Jan01', 'Band 13: Feb01', 'Band 14: Mar01', 'Band 15: Apr01', 'Band 16: May01', 'Band 17: Jun01', 'Band 18: Jul01', 'Band 19: Aug01', 'Band 20: Sep01', 'Band 21: Oct01', 'Band 22: Nov01', 'Band 23: Dec01', 'Band 24: Jan02', 'Band 25: Feb02', 'Band 26: Mar02', 'Band 27: Apr02', 'Band 28: May02', 'Band 29: Jun02', 'Band 30: Jul02', 'Band 31: Aug02', 'Band 32: Sep02', 'Band 33: Oct02', 'Band 34: Nov02', 'Band 35: Dec02', 'Band 36: Jan03', 'Band 37: Feb03', 'Band 38: Mar03', 'Band 39: Apr03', 'Band 40: May03', 'Band 41: Jun03', 'Band 42: Jul03', 'Band 43: Aug03', 'Band 44: Sep03', 'Band 45: Oct03', 'Band 46: Nov03', 'Band 47: Dec03', 'Band 48: Jan04', 'Band 49: Feb04', 'Band 50: Mar04', 'Band 51: Apr04', 'Band 52: May04', 'Band 53: Jun04', 'Band 54: Jul04', 'Band 55: Aug04', 'Band 56: Sep04', 'Band 57: Oct04', 'Band 58: Nov04', 'Band 59: Dec04', 'Band 60: Jan05', 'Band 61: Feb05', 'Band 62: Mar05', 'Band 63: Apr05', 'Band 64: May05', 'Band 65: Jun05', 'Band 66: Jul05', 'Band 67: Aug05', 'Band 68: Sep05', 'Band 69: Oct05', 'Band 70: Nov05', 'Band 71: Dec05', 'Band 72: Jan06', 'Band 73: Feb06', 'Band 74: Mar06', 'Band 75: Apr06', 'Band 76: May06', 'Band 77: Jun06', 'Band 78: Jul06', 'Band 79: Aug06', 'Band 80: Sep06', 'Band 81: Oct06', 'Band 82: Nov06', 'Band 83: Dec06', 'Band 84: Jan07', 'Band 85: Feb07', 'Band 86: Mar07', 'Band 87: Apr07', 'Band 88: May07', 'Band 89: Jun07', 'Band 90: Jul07', 'Band 91: Aug07', 'Band 92: Sep07', 'Band 93: Oct07', 'Band 94: Nov07', 'Band 95: Dec07', 'Band 96: Jan08', 'Band 97: Feb08', 'Band 98: Mar08', 'Band 99: Apr08', 'Band 100: May08', 'Band 101: Jun08', 'Band 102: Jul08', 'Band 103: Aug08', 'Band 104: Sep08', 'Band 105: Oct08', 'Band 106: Nov08', 'Band 107: Dec08', 'Band 108: Jan09', 'Band 109: Feb09', 'Band 110: Mar09', 'Band 111: Apr09', 'Band 112: May09', 'Band 113: Jun09', 'Band 114: Jul09', 'Band 115: Aug09', 'Band 116: Sep09', 'Band 117: Oct09', 'Band 118: Nov09', 'Band 119: Dec09', 'Band 120: Jan10', 'Band 121: Feb10', 'Band 122: Mar10', 'Band 123: Apr10', 'Band 124: May10', 'Band 125: Jun10', 'Band 126: Jul10', 'Band 127: Aug10', 'Band 128: Sep10', 'Band 129: Oct10', 'Band 130: Nov10', 'Band 131: Dec10', 'Band 132: Jan11', 'Band 133: Feb11', 'Band 134: Mar11', 'Band 135: Apr11', 'Band 136: May11', 'Band 137: Jun11', 'Band 138: Jul11', 'Band 139: Aug11', 'Band 140: Sep11', 'Band 141: Oct11', 'Band 142: Nov11', 'Band 143: Dec11', 'Band 144: Jan12', 'Band 145: Feb12', 'Band 146: Mar12', 'Band 147: Apr12', 'Band 148: May12', 'Band 149: Jun12', 'Band 150: Jul12', 'Band 151: Aug12', 'Band 152: Sep12', 'Band 153: Oct12', 'Band 154: Nov12', 'Band 155: Dec12', 'Band 156: Jan13', 'Band 157: Feb13', 'Band 158: Mar13', 'Band 159: Apr13', 'Band 160: May13', 'Band 161: Jun13', 'Band 162: Jul13', 'Band 163: Aug13', 'Band 164: Sep13', 'Band 165: Oct13', 'Band 166: Nov13', 'Band 167: Dec13', 'Band 168: Jan14', 'Band 169: Feb14', 'Band 170: Mar14', 'Band 171: Apr14', 'Band 172: May14', 'Band 173: Jun14', 'Band 174: Jul14', 'Band 175: Aug14', 'Band 176: Sep14', 'Band 177: Oct14', 'Band 178: Nov14', 'Band 179: Dec14', 'Band 180: Jan15', 'Band 181: Feb15', 'Band 182: Mar15', 'Band 183: Apr15', 'Band 184: May15', 'Band 185: Jun15', 'Band 186: Jul15', 'Band 187: Aug15', 'Band 188: Sep15', 'Band 189: Oct15', 'Band 190: Nov15', 'Band 191: Dec15', 'Band 192: Jan16', 'Band 193: Feb16', 'Band 194: Mar16', 'Band 195: Apr16', 'Band 196: May16', 'Band 197: Jun16', 'Band 198: Jul16', 'Band 199: Aug16', 'Band 200: Sep16', 'Band 201: Oct16', 'Band 202: Nov16', 'Band 203: Dec16', 'Band 204: Jan17', 'Band 205: Feb17', 'Band 206: Mar17', 'Band 207: Apr17', 'Band 208: May17', 'Band 209: Jun17', 'Band 210: Jul17', 'Band 211: Aug17', 'Band 212: Sep17', 'Band 213: Oct17', 'Band 214: Nov17', 'Band 215: Dec17', 'Band 216: Jan18', 'Band 217: Feb18', 'Band 218: Mar18', 'Band 219: Apr18', 'Band 220: May18', 'Band 221: Jun18', 'Band 222: Jul18', 'Band 223: Aug18', 'Band 224: Sep18', 'Band 225: Oct18', 'Band 226: Nov18', 'Band 227: Dec18', 'Band 228: Jan19', 'Band 229: Feb19', 'Band 230: Mar19', 'Band 231: Apr19', 'Band 232: May19', 'Band 233: Jun19', 'Band 234: Jul19', 'Band 235: Aug19', 'Band 236: Sep19', 'Band 237: Oct19', 'Band 238: Nov19', 'Band 239: Dec19', 'Band 240: Jan20', 'Band 241: Feb20', 'Band 242: Mar20', 'Band 243: Apr20', 'Band 244: May20', 'Band 245: Jun20', 'Band 246: Jul20', 'Band 247: Aug20', 'Band 248: Sep20', 'Band 249: Oct20', 'Band 250: Nov20', 'Band 251: Dec20', 'Band 252: Jan21', 'Band 253: Feb21', 'Band 254: Mar21', 'Band 255: Apr21', 'Band 256: May21', 'Band 257: Jun21', 'Band 258: Jul21', 'Band 259: Aug21', 'Band 260: Sep21', 'Band 261: Oct21', 'Band 262: Nov21', 'Band 263: Dec21']
target_lai_band_names = ['LAI'+d[-5:] for d in lai_band_names]

# for i in range(1,22):
#   if len(str(i)) ==1:
#     lai_band_names.append('Band 00'+ str(i)+": ")
#   elif len(str(i)) ==2:
#     lai_band_names.append('Band 0'+ str(i)+": ")

time_series_gdf_lai = turn_raster_output_to_gdf(lai_fp, points_shp, band_names_stack=lai_band_names, prefire_months = 120)

# rename bands
renameLAI_dict = {i:j for i,j in zip(lai_band_names, target_lai_band_names)}
time_series_gdf_lai = time_series_gdf_lai.rename(columns=renameLAI_dict)

time_series_gdf_lai.head()

### Read in VCF Data (annual) - to add

In [ ]:


vcf_tree_bandnames = ['PctTree00','PctTree01','PctTree02','PctTree03','PctTree04','PctTree05','PctTree06','PctTree07','PctTree08','PctTree09','PctTree10','PctTree11','PctTree12','PctTree13','PctTree14','PctTree15','PctTree16','PctTree17','PctTree18','PctTree19','PctTree20']
time_series_gdf_vcfTree = turn_raster_output_to_gdf(vcf_tree_fp, points_shp, vcf_tree_bandnames, prefire_months = 120)
time_series_gdf_vcfTree.head()

#### Preprocess VCF Tree Cover Pct DataFrame

In [ ]:
#preprocess VCf Tree cover data

# Assuming you have a Pandas GeoDataFrame called 'gdf'
# Replace 'gdf' with the name of your GeoDataFrame

# Dictionary to map month numbers to their corresponding names
month_mapping = {
    1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
    7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"
}

# List to store the columns to be deleted
columns_to_delete = []

# Loop through each column in the GeoDataFrame
for column in time_series_gdf_vcfTree.columns:
    # Check if the column name starts with "PctTree" (assuming all such columns are to be duplicated)
    if column.startswith("PctTree"):
        # Get the year from the column name (assuming it is the last two characters of the column name)
        year = column[-2:]

        # Loop through each month (1 to 12) and create a new column for each
        for month in range(1, 13):
            # Create the new column name for the current year
            new_column_name = column.replace("PctTree", "Tree" + month_mapping[month])

            # Copy the data from the original column to the new column for the current year
            time_series_gdf_vcfTree[new_column_name] = time_series_gdf_vcfTree[column]

        # If the current year is 2020, copy the data to the new column for the next year (2021)
        if year == "20":

            for month in range(1, 13):
              # Create the new column name for the current year
              new_column_name = column.replace("PctTree20", "Tree" + month_mapping[month]+"21")

              # Copy the data from the original column to the new column for the current year
              time_series_gdf_vcfTree[new_column_name] = time_series_gdf_vcfTree[column]

        # Add the original column to the list of columns to delete
        columns_to_delete.append(column)

# Delete the original columns
time_series_gdf_vcfTree.drop(columns=columns_to_delete, inplace=True)

# Now the GeoDataFrame 'gdf' will have the original columns duplicated 12 times with the new column names
# "TreeJan00", "TreeFeb00", etc., and the original columns that started with "PctTree" will be deleted.
# Additionally, the 2020 data will be copied into 12 additional columns for 2021.


### Read in fire boundary geojson

In [ ]:
# Read in the fire boundaries, converting the date into a readable format
fire_bounds = gpd.read_file(fire_bounds_fp)#.to_crs(time_series_gdf.crs)#fireboundID, "ForestFires_MLFinal.geojson").to_crs(time_series_gdf.crs)
print("CRS is equal:",fire_bounds.crs==time_series_gdf.crs)
fire_bounds['Ig_Date'] = fire_bounds.Ig_Date.apply(convert_epoch_milli_to_datetime)
fire_bounds_dates = fire_bounds[['Ig_Date','Event_ID', 'geometry']]

#check whether event ID is unique
print(len(fire_bounds_dates['Event_ID']) == len(set(fire_bounds_dates['Event_ID'])))


### Add in the dates from the fire boundaries to the point data

In [ ]:
# Add in the dates from the fire boundaries to the point data
time_series_fire_date = gpd.sjoin(time_series_gdf, fire_bounds_dates, how='inner')
#rename column to prepare for next sjoin
time_series_fire_date = time_series_fire_date.rename(columns={'index_right': 'index_right1'})
#join lai data frame with dynamic stack (ndiv, precip, temp) data frame
time_series_fire_date = gpd.sjoin(time_series_fire_date, time_series_gdf_lai)
time_series_fire_date = time_series_fire_date.rename(columns={'index_right': 'index_right2'})

time_series_fire_date = gpd.sjoin(time_series_fire_date, time_series_gdf_vcfTree)

time_series_fire_date = time_series_fire_date[~time_series_fire_date.index.duplicated(keep='first')]
time_series_fire_date = time_series_fire_date.drop(['index_right'], axis=1)
print(time_series_fire_date['Ig_Date'][0:2])
time_series_fire_date.head()


### Split gdf into variable groups for viz.

In [ ]:
# Split the dataframe into the variables currently stored in it
selectThese =  [i.startswith('precip') or 'Ig_Date' in i or 'Event_ID' in i for i in time_series_fire_date.columns] #bool list
precip = time_series_fire_date[time_series_fire_date.columns[selectThese]]

selectThese =  [i.startswith('temp') or 'Ig_Date' in i  for i in time_series_fire_date.columns] #bool list
temp = time_series_fire_date[time_series_fire_date.columns[selectThese]]

selectThese =  [i.startswith('NDVI') or 'Ig_Date' in i  for i in time_series_fire_date.columns] #bool list
ndvi = time_series_fire_date[time_series_fire_date.columns[selectThese]]

selectThese =  [i.startswith('LAI') or 'Ig_Date' in i for i in time_series_fire_date.columns] #bool list
lai = time_series_fire_date[time_series_fire_date.columns[selectThese]]


selectThese =  [i.startswith('Tree') or 'Ig_Date' in i for i in time_series_fire_date.columns] #bool list
vcf_tree = time_series_fire_date[time_series_fire_date.columns[selectThese]]


In [ ]:
# Put the regular data frames into geodataframes so it can be visually spot checked
precip_gdf = gpd.GeoDataFrame(precip, geometry=time_series_fire_date.geometry, crs=time_series_fire_date.crs)
temp_gdf = gpd.GeoDataFrame(temp, geometry=time_series_fire_date.geometry, crs=time_series_fire_date.crs)
ndvi_gdf = gpd.GeoDataFrame(ndvi, geometry=time_series_fire_date.geometry, crs=time_series_fire_date.crs)
lai_gdf = gpd.GeoDataFrame(lai, geometry=time_series_fire_date.geometry, crs=time_series_fire_date.crs)
vcf_tree_gdf = gpd.GeoDataFrame(vcf_tree, geometry=time_series_fire_date.geometry, crs=time_series_fire_date.crs)


### Viz Data

In [ ]:
# Plot the data types and their spatial extents
fig, axes = plt.subplots(2, 2, figsize=(10,10))
ax1, ax2, ax3, ax4 = axes[0][0], axes[0][1], axes[1][0], axes[1][1]
ax1.set_title("precip")
ax2.set_title("temp")
ax3.set_title("ndvi")
precip_gdf.plot(column="precipJan2000", ax=ax1, markersize=1)
temp_gdf.plot(column="tempJan2000", ax=ax2, markersize=1)
ndvi_gdf.plot(column="NDVIJan2000", ax=ax3, markersize=1)
lai_gdf.plot(column="LAIFeb01", ax=ax4, markersize=1)
# id_gdf.plot(column='Post_ID', ax=ax4, markersize=1)
for ax in axes:
  for a in ax:
    fire_bounds.boundary.plot(ax=a, color='black', linewidth=.5)
fig.suptitle("Plot the first time stamp from each variable")
plt.show()

### Split data into model input  vs. output

- input: pre-fire NDVI, precip, temp

- output: post-fire NDVI


In [ ]:
# Take the datasets created, loop through them, parse the date from the column
# name, and then create a pandas time series for just that variable at each point

#how many months after fire do we want to split model input vs. output?
postfire_mo = 12
prefire_mo = 120 + postfire_mo  #120 = 12 mo/year * 10 years

#will want to change the fire date break point to be later..
precip_series = []
temp_series = []
ndvi_before_series = []
ndvi_after_series = []
lai_before_series = []
lai_after_series = []
vcf_tree_before_series = []
vcf_tree_after_series = []
event_series = []

# Parsing the precip data
for i, row in precip_gdf.iterrows():
    #i is the row index; row is the row value container
    # print(i)
    # print('\n\n\n',row)
    # break
    # Get the dates from the column name
    dates = [parser.parse(j[6:]) for j in row.index[:-3]]
    dates = [d.replace(day=calendar.monthrange(d.year, d.month)[1], hour=23, minute = 59) for d in dates] #format: NDVIFeb2000 - wer're selecting Feb2000

    # Get the fire date
        #increment fire date by 1 month

    fire_date = row.Ig_Date #+ relativedelta(months=+1)
    split_date = fire_date + relativedelta(months = postfire_mo)

    # Get the date of 10 years before the fire (or split date)
    #increment fire date by n months
    fire_date_10_before = split_date - relativedelta(months = prefire_mo )#+ postfire_mo)

    # Put the data into a pandas time series, clip it to the temporal extent, and append to a list
    precip_series.append(pd.Series(row.values[:-3], index=dates, name=i)[fire_date_10_before:split_date].tolist())
    event_series.append(row.values[-2]) #event ID value


# Parsing the temperature data
for i, row in temp_gdf.iterrows():
    # Get the dates from the column name
    dates = [parser.parse(j[4:]) for j in row.index[:-2]]
    dates = [d.replace(day=calendar.monthrange(d.year, d.month)[1], hour=23, minute = 59) for d in dates] #format: NDVIFeb2000 - wer're selecting Feb2000

    # Get the fire date
    fire_date = row.Ig_Date #+ relativedelta(months=+1)
    split_date = fire_date + relativedelta(months = postfire_mo)
    # Get the date of 10 years before the fire
    fire_date_10_before = split_date - relativedelta(months = prefire_mo )#+ postfire_mo)
    #increment fire date by 1 month
    # fire_date = fire_date + relativedelta(months=+1)
    # Put the data into a pandas time series, clip it to the temporal extent, and append to a list
    temp_series.append(pd.Series(row.values[:-2], index=dates, name= i)[fire_date_10_before:split_date].tolist())

# Parsing the NDVI data
for i, row in ndvi_gdf.iterrows():
    # Get the dates from the column name
    dates = [parser.parse(j[4:]) for j in row.index[:-2]] #format: NDVIFeb2000 - wer're selecting Feb2000
    dates = [d.replace(day=calendar.monthrange(d.year, d.month)[1], hour=23, minute = 59) for d in dates] #format: NDVIFeb2000 - wer're selecting Feb2000

    # Get the fire date
    fire_date = row.Ig_Date #+ relativedelta(months=+1)
    split_date = fire_date + relativedelta(months = postfire_mo)
    # Get the date of 10 years before the fire
    fire_date_10_before = split_date - relativedelta(months = prefire_mo )#+ postfire_mo)
    # Get the date of 9 years after the fire (not 10 due to a few fires that this goes to far into the future for)
    fire_date_10_after = split_date + relativedelta(months = 110-postfire_mo)
    # Adding both datasets to seperate lists
    ndvi_before_series.append(pd.Series(row.values[:-2], index=dates, name= i)[fire_date_10_before:split_date].tolist()) #exclusive of fire_date
    ndvi_after_series.append(pd.Series(row.values[:-2], index=dates, name= i)[split_date:fire_date_10_after].tolist()) #inclusive of fire_date



In [ ]:
#parse LAI data
geom_series =[ ]

for i, row in lai_gdf.iterrows():
  
    # Get the dates from the column name
    dates = [parser.parse(j[3:6]+'20'+j[-2:]) for j in row.index[1:-1]] #only using row index [1:-1] to exclude first and last col, which are ig_date and geometry,respectively
    dates = [d.replace(day=calendar.monthrange(d.year, d.month)[1], hour=23, minute = 59) for d in dates] #format: NDVIFeb2000 - wer're selecting Feb2000

    # Get the fire date
    fire_date = row.Ig_Date #+ relativedelta(months=+1)
    split_date = fire_date + relativedelta(months = postfire_mo)
    # Get the date of 10 years before the fire
    fire_date_10_before = split_date - relativedelta(months = prefire_mo )#+ postfire_mo)
    # Get the date of 9 years after the fire (not 10 due to a few fires that this goes to far into the future for)
    fire_date_10_after = split_date + relativedelta(months = 110-postfire_mo)
    # Adding both datasets to seperate lists
    lai_before_series.append(pd.Series(row.values[1:-1], index=dates, name= i)[fire_date_10_before:split_date].tolist()) #exclusive of fire_date
    lai_after_series.append(pd.Series(row.values[1:-1], index=dates, name= i)[split_date:fire_date_10_after].tolist()) #inclusive of fire_date
    geom_series.append(row.geometry)




In [ ]:
plt.plot(lai_before_series[96])
plt.plot(lai_after_series[96])


In [ ]:
#parse VCF Tree data

for i, row in vcf_tree_gdf.iterrows():
    # Get the dates from the column name
    dates = [parser.parse(j[4:7]+'20'+j[-2:]) for j in row.index[1:-1]] #only using row index [1:-1] to exclude first and last col, which are ig_date and geometry,respectively
    dates = [d.replace(day=calendar.monthrange(d.year, d.month)[1], hour=23, minute = 59) for d in dates] #format: NDVIFeb2000 - wer're selecting Feb2000

    # print(row.index[1][4:7]+'20'+row.index[1][-2:])
    # break
    # Get the fire date
    fire_date = row.Ig_Date #+ relativedelta(months=+1)
    split_date = fire_date + relativedelta(months = postfire_mo)
    # Get the date of 10 years before the fire
    fire_date_10_before = split_date - relativedelta(months = prefire_mo )#+ postfire_mo)
    # Get the date of 9 years after the fire (not 10 due to a few fires that this goes to far into the future for)
    fire_date_10_after = split_date + relativedelta(months = 110-postfire_mo)
    # Adding both datasets to seperate lists
    vcf_tree_before_series.append(pd.Series(row.values[1:-1], index=dates, name= i)[fire_date_10_before:split_date].tolist()) #exclusive of fire_date
    vcf_tree_after_series.append(pd.Series(row.values[1:-1], index=dates, name= i)[split_date:fire_date_10_after].tolist()) #inclusive of fire_date




In [ ]:

# reorganize = pd.DataFrame(list(zip(*[ lai_before_series,event_series])),
#                           columns = ['lai','Event_ID'])
# print(reorganize.head())
# reorganize_gdf = gpd.GeoDataFrame(reorganize,
#                                   geometry = geom_series,
#                                   crs = time_series_fire_date.crs)


# time_series_fire_date.head()
# #this is fine too
# save_this = reorganize_gdf[['Event_ID','geometry']].copy()
# save_this.to_file("next_test56.shp",geometry='geometry')

In [ ]:

#check that theyre all the same length
print(len(ndvi_after_series), len(ndvi_after_series[1]))#[1].shape)
print(len(ndvi_before_series), len(ndvi_before_series[1]))#[1].shape)
print(len(lai_after_series), len(lai_after_series[1]))#[1].shape)


for i in range(len(ndvi_before_series)):
  if len(precip_series[i]) !=prefire_mo or len(temp_series[i]) !=prefire_mo or len(ndvi_before_series[i])!=prefire_mo or len(lai_before_series[i])!= prefire_mo or len(vcf_tree_before_series[i]) !=prefire_mo:
    print(i)
    raise
  if np.isnan(precip_series[i][-1]) and not np.isnan(precip_series[i][1]):
    print("NAN", precip_series[i][-1])
    raise

for i in range(len(ndvi_after_series)):
  if len(ndvi_after_series[i])!=110-postfire_mo or len(lai_after_series[i])!=110-postfire_mo or (np.isnan(ndvi_after_series[i][-1]) and not np.isnan(ndvi_after_series[i][1])):
    print(i, len(ndvi_after_series[i]), ndvi_after_series[i][-1])
    raise


### Re-create gdf where columns contain listed timeseries

In [ ]:
# Reingests all of the data into a geodataframe that has the time series for
#each variable stored in each cell, and the ndvi split up before and after fire date

Ig_Series = time_series_fire_date['Ig_Date']
Id_Series = time_series_fire_date['Event_ID']#event_series (also works) #time_series_fire_date['Event_ID'] #event series is list of fires for each row in df (so repeating fire ids)
#Note on Event_ID [from GEE]: Unique identifier for each event (21 characters). Calculated from source data (ICS209, FedFire, etc.) ;
#   each time an event is created or updated using the state, lat/long coordinates (ig_lat, ig_long). Note that for longitudes less than 100° a leading zero is added to maintain 21 characters.

reorganize = pd.DataFrame(list(zip(*[precip_series, temp_series, ndvi_before_series, lai_before_series, vcf_tree_before_series,
                                      ndvi_after_series, lai_after_series,vcf_tree_after_series,
                                      Ig_Series, Id_Series, time_series_fire_date.geometry])),
                          columns = ['precip', 'temp', 'ndvi_before', 'lai_before', 'vcf_tree_before',
                                     'ndvi_after', 'lai_after', 'vcf_tree_after',
                                     'Ig_Date','Event_ID', 'geom'])
reorganize_gdf = gpd.GeoDataFrame(reorganize,
                                  geometry = 'geom',#time_series_fire_date.geometry,
                                  crs = time_series_fire_date.crs)



In [ ]:

temp_cols = [i for i in reorganize_gdf.columns if 'lai_b' in i]
sdfg=reorganize_gdf[temp_cols].iloc[96].tolist()
plt.plot(sdfg[0])

temp_cols = [i for i in reorganize_gdf.columns if 'lai_a' in i]
sdfg=reorganize_gdf[temp_cols].iloc[96].tolist()
plt.plot(sdfg[0])
#.plot(label='LAI',color='green')
# plt.legend(loc='upper right')#

### Read static-var rasters

In [ ]:
# Loop through all static rasters and extract the point data needed for them
# There are 4 rasters that cover different spatial extents of Colorado. The full state was too big to export as one raster
static_rasters =[static_fp1, static_fp2, static_fp3, static_fp4]
#NOTE******** this static stack contains the wrong version of burn severity 
static_gdfs = []
for raster in static_rasters:
    print(raster, 'is processing..')
    print(os.path.exists(raster))
    static_gdfs.append(turn_raster_output_to_gdf_STATIC(raster, time_series_fire_date, ['slope', 'chili', 'elevation', 'aspect', 'severity', 'mtpi'],prefire_mo))
    


### Add burn severity seperately from static rasters

In [ ]:
#explicitly add in burn severity bc this layer has a resolution-based issue and thus leads to no data

# Loop through all static rasters and extract the point data needed for them
# There are 4 rasters that cover different spatial extents of Colorado. The full state was too big to export as one raster
burnsev_rasters_prj = [os.path.join(burn_main_dir,tif) for tif in os.listdir(burn_main_dir) if tif.endswith('.tif')]
burnsev_rasters_withPoints = [] #store which rasters actually do have points in them (for burn sev this should be all of them)
burnsev_gdfs = []

for raster in burnsev_rasters_prj:
    print(raster, 'is processing..')
    to_add = turn_raster_output_to_gdf_STATIC(raster, time_series_fire_date, ['temp_sev'],prefire_mo)
    if to_add is not None:
        burnsev_rasters_withPoints.append(raster)
    burnsev_gdfs.append(to_add)

### Add static variables to final gdf

In [ ]:
# Append all geodataframes from static variables into one geodataframe
static_gdf = static_gdfs[0].append(static_gdfs[1]).append(static_gdfs[2]).append(static_gdfs[3]) #instead of append, may need to use pd.concat in future

print('static_gdf',static_gdf.shape)
static_gdf = pd.concat(static_gdfs[g] for g in range(len(static_gdfs)))
burnsev_gdf = pd.concat(burnsev_gdfs[g] for g in range(len(burnsev_gdfs)))

# Run a spatial join to combine the time series variables and static variables by point
all_gdf = gpd.sjoin(static_gdf, reorganize_gdf)#,how='right')
print('allgdf init shp:',all_gdf.shape)
all_gdf = all_gdf.rename(columns={'index_left': 'index_lef1', 'index_right':'index_right1'})

#Drop duplicate points (based on geometry)
all_gdf.drop_duplicates(subset='geometry',keep='first',inplace=True)
print(all_gdf.shape,'is shp after drop duplicates (based on geometry field)')

all_gdf = gpd.sjoin(all_gdf, burnsev_gdf) # need ot add all form bursev_gdfs list
print('all gdf w burn severity shape:',all_gdf.shape)

all_gdf['severity'] = all_gdf['temp_sev'] #assign original severity column with true severity values 
all_gdf = all_gdf.drop('temp_sev', axis=1) #drop the true severity values stored in temporary (now duplicate) column
print(all_gdf.shape,'is shape w severity before dropping duplicate geoms')
all_gdf=all_gdf.drop_duplicates(subset='geometry',keep='first')

print(all_gdf.shape,'is shp after drop duplicates (based on geometry field)')
all_gdf.head()


In [ ]:
# dup=all_gdf['geometry'].duplicated()
# mask = all_gdf['severity'][dup] #(1135,)
# # all_gdf[dup]


# print(mask.shape)

# mask.index

# for i in mask.index:#range(120):
# #     print(mask[i])
# #     print(all_gdf['severity'][i])
#     print(all_gdf.loc[all_gdf.index[i], 'severity'])

In [ ]:
band_names = []
years = list(range(2000, 2022))
months = list(calendar.month_name[1:])
vars = ['precip', 'temp', 'NDVI', 'LAI', 'VCF_tree']

for year in years:
  for month in months:
    for var in vars:
      band_names.append(var + month[:3] + str(year))

### Remove rows where there is no data in the static variable columns (17 samples)

In [ ]:
all_gdf_cp =  all_gdf.reset_index(drop=True)


dropcnt=0
# Function to calculate nanmean of a list and handle None case
def safe_nanmean(lst):
    return np.nanmean(lst) if not all(np.isnan(lst)) else None

store_inx=[]
# Iterate through rows in the GeoDataFrame
for index, row in all_gdf_cp.iterrows():

    # Calculate nanmean of the 'slope' column for each row
    nanmean_value = safe_nanmean(row['slope'])
    
    # Check if nanmean is None (all values were nan)
    if nanmean_value is None:
        dropcnt+=1
        # Drop the row if nanmean is None
        all_gdf_cp.drop(index, inplace=True)

# Now 'gdf' will not contain rows where nanmean of 'slope' is None
dropcnt



In [ ]:
print(all_gdf.shape)
print(all_gdf_cp.shape) # should be 998,19 ( bc 17 less than 1015)

all_gdf = all_gdf_cp.copy()
# all_gdf_cp

### Export .GDF
(export to shapefiles - first the full/raw data (first element though) and in the next cell export the variable averages)

In [ ]:

#Export all_gdf 
all_gdf.to_pickle("all_gdf.pkl")

In [ ]:
# Function that takes a list and returns its first element
def list_zero(x):
    if type(x)==list:
        return x[0]
#     else:
#         print('none','is x')

def create_str(in_list):
    return "_".join([str(k) for k in in_list])

static_single = all_gdf.copy()
dyn_vars = ['precip','temp', 'ndvi_before', 'lai_before', 'vcf_tree_before','ndvi_after', 'lai_after', 'vcf_tree_after']
static_vars = ['slope', 'chili', 'elevation', 'aspect', 'severity', 'mtpi']


# For each static variable, map it to its first element
for var in static_vars:
    print(var)
    static_single[var] = static_single[var].map(list_zero)

for var in dyn_vars:
    print(var)
    static_single[var] = static_single[var].map(create_str)
    
#     print(static_single[var])
print('done')
# Perform a spatial join between static_single and time_series_fire_date, and create new columns for Year, Month, and Day based on Ig_Date
new_join = static_single#gpd.sjoin(static_single, time_series_fire_date)
new_join['Year'] = new_join['Ig_Date'].map(lambda x: x.year)
new_join['Month'] = new_join['Ig_Date'].map(lambda x: x.month)
new_join['Day'] = new_join['Ig_Date'].map(lambda x: x.day)

# Drop the Ig_Date column from new_join
new_join = new_join.drop('Ig_Date', axis=1)

fixed_band_names = {}

# Rename bands to be smaller names so it's shapefile friendly
for i in band_names:
  if i.startswith('N'):
    fixed_band_names[i] = i.replace('NDVI', 'N')
  elif i.startswith('t'):
    fixed_band_names[i] = i.replace('temp', 'T')
  elif i.startswith('p'):
    fixed_band_names[i] = i.replace('precip', 'P')
  elif i.startswith('L'):
    fixed_band_names[i] = i.replace('LAI', 'L')
  elif i.startswith('V'):
    fixed_band_names[i] = i.replace('VCF_tree', 'F') #replace w F for forest since T is used
  else:
    print(i)

new_join = new_join.rename(columns=fixed_band_names)

# Write to a shapefile
print("writing new shapefile: Quant_Fire_Data_list")

#uncomment this to create shapefile - this takes a while to run
new_join.to_file("Quant_Fire_Data_list.shp")


In [ ]:
xlpath='LSTM_Data.csv'

all_gdf_toexcel = all_gdf.rename(columns={'ndvi_before': 'pre_ndvi', 
                                          'lai_before':'pre_lai', 
                                          'ndvi_after': 'post_ndvi', 
                                          'lai_after':'post_lai',
                                          'vcf_tree_before':'pre_tree',
                                         'vcf_tree_after':'post_tree'})


all_gdf_toexcel['x'] = all_gdf_toexcel['geometry'].x
all_gdf_toexcel['y'] = all_gdf_toexcel['geometry'].y

# all_gdf_toexcel['ig_month'] = all_gdf_toexcel['Ig_Date'].month
all_gdf_toexcel['ig_month'] = [this_day.month for this_day in all_gdf_toexcel['Ig_Date']]
all_gdf_toexcel['ig_day'] = [this_day.day for this_day in all_gdf_toexcel['Ig_Date']]
all_gdf_toexcel['ig_year'] = [this_day.year for this_day in all_gdf_toexcel['Ig_Date']]

all_gdf_toexcel = all_gdf_toexcel.drop(['index_right1','index_right'], axis=1)

# all_gdf_toexcel= pd.DataFrame(all_gdf_toexcel)

# print(all_gdf_toexcel.head()[:1])

# all_gdf_toexcel.to_file(xlpath)#, driver='excel')
all_gdf_toexcel.to_csv(xlpath)#, index=0)
# pd.__version__ #'1.5.3'

#read back in:

#instead of going through all the data preprocessing above - just use this excel sheet
new_gdfx = pd.read_csv(xlpath)
new_gdfx.head()

In [ ]:
#NOTE: 17 points are outside of the static variable raster so have no data there

fire_list ={event_id:{} for event_id in set(all_gdf['Event_ID'])}


list_variables = ['slope', 'chili', 'elevation', 'aspect', 'severity', 'mtpi',
                    'precip', 'temp', 'ndvi_before','lai_before','vcf_tree_before',
                   'ndvi_after','lai_after', 'vcf_tree_after']#,'geometry']

count_errors = 0
list_error_fires=[]
#iterate through rows of gdf
for index, row in all_gdf.iterrows():
    event_id = row["Event_ID"]
    for variable in list_variables:
        if variable in fire_list[event_id].keys():
            try:
                
                fire_list[event_id][variable].extend(row[variable]) #here
            except:
                count_errors+=1
                list_error_fires.append(event_id)
                
        else:
            fire_list[event_id][variable] = list(row[variable]) 
            
print('num errors',count_errors/4) # divide by 4 bc thats how many layers are in static bands (minus burn severity which was on its own)
print('error ids:', set(list_error_fires))

#want dataframe with rows = event id and columns = "slope_mean", "aspect_std", etc.
fire_stats = pd.DataFrame()
# Define the data as a dictionary
# data = {'mean': [1, 2, 3], 'std': [0.1, 0.2, 0.3]}

# Create the DataFrame
rownames = [i+'_'+j for i in ['mean','std','min','max'] for j in list_variables]
empty_array=np.zeros((len(fire_list),len(rownames)))
df = pd.DataFrame(empty_array, index=fire_list, columns=rownames)

this_idx=-1
for event_id, variable in fire_list.items():
    print(event_id,"is fire")
    #iterate through fires
    #event id is ID of fire e.g. CO3872810860220100712
    #variable is a dictionary, where keys are like 'slope','mtpi' and values are list of list of values
    for feat,stat_list in variable.items():
        #feat is something like 'slope', 'mtpi'
        this_mean = round(np.nanmean(stat_list),3)
        this_std = round(np.nanstd(stat_list),3)
        this_min = round(np.min(stat_list),3)
        this_max = round(np.max(stat_list),3)
        df.loc[[event_id], ['mean_{}'.format(feat)]] = this_mean
        df.loc[[event_id], ['std_{}'.format(feat)]] = this_std
        df.loc[[event_id], ['min_{}'.format(feat)]] = this_min
        df.loc[[event_id], ['max_{}'.format(feat)]] = this_max

df.to_csv("per_fire_summary_stats.csv")


## Split training and test fires

In [ ]:
# Run a custom test/train split so that test is made of completely seperate fires then train
test = []
train = []
train_id_dict = {}
test_id_dict = {}
total_points, test_fire_num, train_fire_num = 0, 0, 0
# Group by date loops through the dataframe by fire (via ignition date)
for group in all_gdf.groupby('Event_ID'):
  # Check how many points are already in the test dataset. Setting this to 700 results in a roughly 80/20 split
  if total_points < 790:
    print('train',len(group[1]))
    range_g = str(total_points) + '-' + str(total_points+len(group[1])-1)
    total_points += len(group[1])
    train.append(group[1][['slope', 'chili', 'elevation', 'aspect', 'severity',
                           'mtpi', 'precip', 'temp', 'ndvi_before','lai_before', 'vcf_tree_before',
                           'ndvi_after','lai_after','vcf_tree_after','geometry', 'Event_ID']])
    train_id_dict[group[1]['Event_ID'].values[0]] = [len(group[1]), range_g]
    train_fire_num += 1

  else:
    print(len(group[1]))
    range_g = str(total_points) + '-' + str(total_points+len(group[1])-1)
    total_points += len(group[1])
    test.append(group[1][['slope', 'chili', 'elevation', 'aspect', 'severity',
                          'mtpi', 'precip', 'temp', 'ndvi_before','lai_before','vcf_tree_before',
                          'ndvi_after','lai_after','vcf_tree_after','geometry','Event_ID']])
    test_fire_num += 1
    test_id_dict[group[1]['Event_ID'].values[0]] = [len(group[1]), range_g]


In [ ]:
# code to randomly select 6 fires for test and 23 for train
test = []
train = []
validation = []

id_index = list(all_gdf.columns).index('Event_ID') # get index value with which to get event_id from each row (should be 17)

total_points, test_fire_num, train_fire_num = 0, 0, 0

all_ids = list(set(train_id_dict.keys())) + list(set(test_id_dict.keys()))


# train_id, test_id, t0, t1 = train_test_split(all_ids, all_ids, test_size = .15, random_state = 3)
# train_id, val_id, t0, t1 = train_test_split(train_id, train_id, test_size = .15, random_state = 3)
# del t0,t1

# #setting this ***manually*** b/c each time I reconnected colab runtime even with same random_state, split was not reproducable
train_id = ['NM3692010445620110612','CO3817710507320121023', 'CO3970610755420100628', 'CO3843610898920120525', 'CO3872810860220100712', 'CO4036110556320121009', 'UT3967210907720100826', 'CO3888410493320120623', 'CO3740210724320120513', 'CO3825210820820110803', 'UT3977510922020120629', 'CO4058910540420120609', 'CO3812310818320100522', 'CO4054610858420100720', 'CO4053710538120110401', 'CO3823310568320110612', 'CO3976210526320110320', 'CO3925110844620110807', 'CO3709810719920101012', 'CO3781310546020100606']
val_id = ['CO3726810830320120622', 'CO3935510767920100507', 'CO4005110538520100906', 'CO3943610521720120326']
test_id = ['CO3747210346920110607', 'NM3700010423620110526', 'NM3696310515520100523', 'CO3741010757920121016', 'CO3894510543620120617'] #removed: 'CO3740210724320120513'
#NOTE: swapped little sand fire into test and track fire into train! 11-22-23

print("LENs:",len(train_id), len(val_id), len(test_id))

print(train_id)
print(val_id)
print(test_id)

print("# of Train Fires:", len(train_id))
print("# of Validation Fires:", len(val_id))
print('# of Test Fires:', len(test_id))

#create dict of lists where each sublist contains all points in n test fire
testfire_dict = {}

for row in all_gdf.iterrows():

  if row[1][id_index] in train_id:

      train.append(row[1][['slope', 'chili', 'elevation', 'aspect', 'severity', 'geometry','Event_ID',
                             'mtpi', 'precip', 'temp', 'ndvi_before','lai_before','vcf_tree_before',
                            'ndvi_after','lai_after', 'vcf_tree_after']])
  elif row[1][id_index] in val_id:
      validation.append(row[1][['slope', 'chili', 'elevation', 'aspect', 'severity', 'geometry','Event_ID',
                             'mtpi', 'precip', 'temp', 'ndvi_before', 'lai_before', 'vcf_tree_before',
                                'ndvi_after','lai_after','vcf_tree_after']])
  elif row[1][id_index] in test_id:
      test.append(row[1][['slope', 'chili', 'elevation', 'aspect', 'severity', 'geometry','Event_ID',
                          'mtpi', 'precip', 'temp', 'ndvi_before', 'lai_before','vcf_tree_before',
                          'ndvi_after','lai_after','vcf_tree_after']])
      if row[1][id_index] in testfire_dict.keys():
        testfire_dict[row[1][id_index]].append(row[1][['slope', 'chili', 'elevation', 'aspect', 'severity',
                          'mtpi', 'precip', 'temp', 'ndvi_before', 'lai_before','vcf_tree_before',
                                                 'ndvi_after','lai_after','vcf_tree_after']])
      else:
        testfire_dict[row[1][id_index]] = [row[1][['slope', 'chili', 'elevation', 'aspect', 'severity',
                          'mtpi', 'precip', 'temp', 'ndvi_before', 'lai_before','vcf_tree_before',
                                             'ndvi_after','lai_after','vcf_tree_after']]]
  else:
    print("ERROR: point not assigned to set:",row[1][id_index])

print("# points in training dataset:", len(train))
print("# points in validation dataset:", len(validation))

print("# points in test dataset:", len(test))
print("Train/Val/Test Split:", len(train)/(len(test)+len(train)+len(validation)), '/',\
      len(validation)/(len(test)+len(train)+len(validation)),'/',len(test)/(len(test)+len(train)+len(validation)))

test_df = pd.DataFrame(test)
train_df = pd.DataFrame(train)
val_df = pd.DataFrame(validation)

testfire_df = {}
for key,val in testfire_dict.items():
  new_val = pd.DataFrame(val)
  testfire_df[key] = new_val


In [ ]:

temp_cols = [i for i in test_df.columns if 'lai_a' in i]
plt.plot(test_df[temp_cols].iloc[96].tolist()[0])

temp_cols = [i for i in test_df.columns if 'lai_b' in i]
plt.plot(test_df[temp_cols].iloc[96].tolist()[0])
# .plot(label='LAI',color='green')
# plt.legend(loc='upper right')

In [ ]:
# create table that has each fire, # of points in the fire bound, and whether
# fire is designated as training or test
#and at the bottom, the total # of training and test points

#These parameters must be 2D lists, in which the outer lists define the rows and
#the inner list define the column values per row.
train_lab = ["Train" for i in range(len(train_id_dict))]
test_lab = ["Test" for i in range(len(test_id_dict))]

# range_pt_ids =[i[1] for i in list(train_id.values)]
num_pts_list = [i[0] for i in list(train_id_dict.values())]+[i[0] for i in list(test_id_dict.values())]
range_pts_list = [i[1] for i in list(train_id_dict.values())]+[i[1] for i in list(test_id_dict.values())]
print(len(range_pts_list),len(num_pts_list))
print(sum(num_pts_list),"is total number of points")
print("Train fires:",list(train_id_dict.keys()))
print("Test fires:",list(test_id_dict.keys()))


table_list =  pd.DataFrame({"Set":train_lab+test_lab,
                            "MTBS Fire ID": list(train_id_dict.keys())+list(test_id_dict.keys()),
                            "Num. Points (pre null filter)": num_pts_list
                            # "Range Point IDs":range_pts_list
                            })
table_list
# col_labels = ["MTBS Fire ID","# Points","Set (Train or Test)"]
# row_labels = [""]



In [ ]:
train_id = ['NM3692010445620110612','CO3817710507320121023', 'CO3970610755420100628', 'CO3843610898920120525', 'CO3872810860220100712', 'CO4036110556320121009', 'UT3967210907720100826', 'CO3888410493320120623', 'CO3740210724320120513', 'CO3825210820820110803', 'UT3977510922020120629', 'CO4058910540420120609', 'CO3812310818320100522', 'CO4054610858420100720', 'CO4053710538120110401', 'CO3823310568320110612', 'CO3976210526320110320', 'CO3925110844620110807', 'CO3709810719920101012', 'CO3781310546020100606']
val_id = ['CO3726810830320120622', 'CO3935510767920100507', 'CO4005110538520100906', 'CO3943610521720120326']
test_id = ['CO3747210346920110607', 'NM3700010423620110526', 'NM3696310515520100523', 'CO3741010757920121016', 'CO3894510543620120617'] #removed: 'CO3740210724320120513'
#NOTE: swapped little sand fire into test and track fire into train! 11-22-23

for i in val_id:
    if i in test_id or i in train_id:
        print(i)

## Viz train vs. test fire bounds

In [ ]:

#note that there are 29 IDs here vs. 28 fires that data are split on bc one fire crosses the CO/UT border
#and thus has 2 dif Event_ID values for it

#seperate fires from fire_bounds in training set and test set

list_train_id = list(set(train_id))
list_val_id = list(set(val_id))
list_test_id = list(set(test_id))

mask = fire_bounds_dates['Event_ID'].isin(list_train_id)
mask_val = fire_bounds_dates['Event_ID'].isin(list_val_id)
mask_test = fire_bounds_dates['Event_ID'].isin(list_test_id)

train_fire_bounds = fire_bounds_dates[mask]
val_fire_bounds = fire_bounds_dates[mask_val]
test_fire_bounds = fire_bounds_dates[mask_test]


print(len(train_fire_bounds), len(val_fire_bounds), len(test_fire_bounds))

In [ ]:

fig, ax = plt.subplots(figsize=(8, 8))
train_fire_bounds.boundary.plot(ax=ax, color='blue', linewidth=1, label='Train')
val_fire_bounds.boundary.plot(ax=ax, color='green', linewidth=1, label='Validation')
test_fire_bounds.boundary.plot(ax=ax, color='orange', linewidth=1, label='Test')
plt.title("Fire Boundaries")
plt.legend(loc='upper right')
plt.savefig("Training_vs_Test_Fire_Bounds",dpi=300)


## Fill NaN values in NDVI_Before series in dataframes

In [ ]:
# forward-fill NaN values in ndvi_before in dataframes

for list_i in range(len(test_df.ndvi_before.values)):
  df_temp = pd.Series(test_df.ndvi_before.values[list_i])
  new_l = df_temp.fillna(method='ffill')
  test_df.ndvi_before.values[list_i] = list(new_l)

for list_i in range(len(train_df.ndvi_before.values)):
  df_temp = pd.Series(train_df.ndvi_before.values[list_i])
  new_l = df_temp.fillna(method='ffill')
  train_df.ndvi_before.values[list_i] = list(new_l)

for list_i in range(len(val_df.ndvi_before.values)):
  df_temp = pd.Series(val_df.ndvi_before.values[list_i])
  new_l = df_temp.fillna(method='ffill')
  val_df.ndvi_before.values[list_i] = list(new_l)


In [ ]:
#save test df
# save train df
# save val df
#NOTE: to run this, need to make sure you don't delete the "geometry" fields in earlier attribute lists

test_df_limited=test_df[['Event_ID','geometry']].copy() #may not work when including 'Event_ID'
test_gdf = gpd.GeoDataFrame(test_df_limited, geometry='geometry')
test_gdf.to_file('test_df_points.shp')


train_df_limited=train_df[['Event_ID','geometry']].copy()
train_gdf = gpd.GeoDataFrame(train_df_limited, geometry='geometry')
train_gdf.to_file('train_df_points.shp')


val_df_limited=val_df[['Event_ID','geometry']].copy()
val_gdf = gpd.GeoDataFrame(val_df_limited, geometry='geometry')
val_gdf.to_file('val_df_points.shp')


### Subset out predictor columns

In [ ]:
# Read test and train data frames into numpy arrays

use_these_bands = ['slope' ,'chili', 'elevation', 'aspect', 'mtpi', 'precip', 
                   'temp', 'ndvi_before','lai_before', 'severity', 'vcf_tree_before']#-> replaced w severity
#should be same at feat_names in ablation study loop


#should be same at feat_names in ablation study loop

x_test_names_forFig = test_df['Event_ID'].values.tolist()

print(pd.DataFrame(test_df[use_these_bands]).head())#.columns)
x_test_df = test_df[use_these_bands].values.tolist()
x_train_df = train_df[use_these_bands].values.tolist()
x_val_df = val_df[use_these_bands].values.tolist()

y_test_df1 = test_df[['ndvi_after']].values.tolist()
y_train_df1 = train_df[['ndvi_after']].values.tolist()
y_val_df1 = val_df[['ndvi_after']].values.tolist()

y_test_df2 = test_df[['lai_after']].values.tolist()
y_train_df2 = train_df[['lai_after']].values.tolist()
y_val_df2 = val_df[['lai_after']].values.tolist()


y_test_df3 = test_df[['vcf_tree_after']].values.tolist()
y_train_df3 = train_df[['vcf_tree_after']].values.tolist()
y_val_df3 = val_df[['vcf_tree_after']].values.tolist()

test_geom = test_df[['geometry', 'Event_ID']].values.tolist()


In [ ]:
#create copy of test data to store IDs
x_test_df_storeIDs = test_df[['Event_ID']].values#.tolist() #assumes no values are removed in next step

## Convert to arrays 

In [ ]:
#NOTE: as of 2/7/23, NDVI_before is the limiting factor here
x_test_names_forFig =np.array(x_test_names_forFig)[~np.isnan(np.array(x_test_df)).any(axis=(1,2))]

X_test = np.array(x_test_df)[~np.isnan(np.array(x_test_df)).any(axis=(1,2))]
X_train = np.array(x_train_df)[~np.isnan(np.array(x_train_df)).any(axis=(1,2))]
X_val = np.array(x_val_df)[~np.isnan(np.array(x_val_df)).any(axis=(1,2))]

y_test1 = np.array(y_test_df1)[~np.isnan(np.array(x_test_df)).any(axis=(1,2))]
y_train1=  np.array(y_train_df1)[~np.isnan(np.array(x_train_df)).any(axis=(1,2))]
y_val1 = np.array(y_val_df1)[~np.isnan(np.array(x_val_df)).any(axis=(1,2))]

y_test2 = np.array(y_test_df2)[~np.isnan(np.array(x_test_df)).any(axis=(1,2))]
y_train2= np.array(y_train_df2)[~np.isnan(np.array(x_train_df)).any(axis=(1,2))]
y_val2 = np.array(y_val_df2)[~np.isnan(np.array(x_val_df)).any(axis=(1,2))]


y_test3, y_train3, y_val3 = np.array(y_test_df3)[~np.isnan(np.array(x_test_df)).any(axis=(1,2))], np.array(y_train_df3)[~np.isnan(np.array(x_train_df)).any(axis=(1,2))],np.array(y_val_df3)[~np.isnan(np.array(x_val_df)).any(axis=(1,2))]

test_geom = np.array(test_geom)[~np.isnan(np.array(x_test_df)).any(axis=(1,2))]

#reformat labels
x_test_list_storeIDs = np.array(x_test_df_storeIDs)#[~np.isnan(np.array(x_test_df_storeIDs)).any(axis=(1,2))]
print("label shp:", x_test_list_storeIDs.shape)
print('X Val:',X_val.shape, 'y val:',y_val1.shape)
print('X_train:',X_train.shape, 'y train:', y_train1.shape)
print('X Test:',X_test.shape, 'y test:',y_test1.shape)
print("Note: shape format is #samples, #features, # of timesteps\n")

tot = y_train1.shape[0]+y_test1.shape[0]+y_val1.shape[0]
print("train/val/test point split",y_train1.shape[0]/tot,"/",y_val1.shape[0]/tot,"/",y_test1.shape[0]/tot )

#check: have we removed points?
print("prev shape of test was:",x_test_df_storeIDs.shape) #first shape 0 should match


In [ ]:
plt.plot(X_train[0,9,:], color='red', label = "vcf")
plt.plot(X_test[0,10,:], label="severity ")

In [ ]:
plt.plot(y_train2[80][0], color='green')

plt.plot(y_test2[7][0])

## Remove 255s; divide by 10k; Standardize Data

### Replace lai values of 255 with 0

In [ ]:
X_train[:,8,:][X_train[:,8,:] == 255] = 0
X_val[:,8,:][X_val[:,8,:] == 255] = 0
X_test[:,8,:][X_test[:,8,:] == 255] = 0

y_train2[:,0,:][y_train2[:,0,:] == 255] = 0
y_test2[:,0,:][y_test2[:,0,:] == 255] = 0
y_val2[:,0,:][y_val2[:,0,:] == 255] = 0

In [ ]:
y_test2.shape

### Divide x_train_ndvi by 10k 
To get to original value before GEE data scaling

In [ ]:
X_train[:,7,:] = X_train[:,7,:]/10000
X_test[:,7,:] = X_test[:,7,:]/10000
X_val[:,7,:] = X_val[:,7,:]/10000

y_train1 = y_train1/10000
y_test1 = y_test1/10000
y_val1 = y_val1/10000


X_train[:,8,:] = X_train[:,8,:]*0.1
X_test[:,8,:] = X_test[:,8,:]*0.1
X_val[:,8,:] = X_val[:,8,:]*0.1

y_train2 = y_train2*0.1
y_test2 = y_test2*0.1
y_val2 = y_val2*0.1



### Standardize

In [ ]:
# Standardize data by hand because StandarScalar doesn't take multidimensional rasters
#NOTE this is for when we have three target vars ********************

n_samples, n_feats, n_tsteps =  X_train.shape

X_train = np.transpose(X_train,axes=(0,2,1))#reshape(n_samples, n_tsteps, n_feats)
print(X_train.shape,"is new shp")

X_train = np.array(X_train)
mean_value = np.mean(X_train, axis= (tuple(range(X_train.ndim-1))))
std_val = np.std(X_train, axis= (tuple(range(X_train.ndim-1))))
print("NDVI mean", mean_value[7])
print("LAI mean", mean_value[8])
print("NDVI std", std_val[7])
print("lai std", std_val[8])



print("orig min of ndvi:", np.min(X_train,axis= (tuple(range(X_train.ndim-1))))[7])
print("orig max of ndvi:", np.max(X_train,axis= (tuple(range(X_train.ndim-1))))[7])

print("orig min of lai:", np.min(X_train,axis= (tuple(range(X_train.ndim-1))))[8])
print("orig max of lai:", np.max(X_train,axis= (tuple(range(X_train.ndim-1))))[8])


#subtract mean and then divide by stdev

X_test = np.transpose(X_test,axes=(0,2,1))#reshape(n_samples, n_tsteps, n_feats)
X_val = np.transpose(X_val,axes=(0,2,1))#reshape(n_samples, n_tsteps, n_feats)

y_train1 = np.transpose(y_train1,axes=(0,2,1))
y_test1 =np.transpose(y_test1,axes=(0,2,1))
y_val1 = np.transpose(y_val1,axes=(0,2,1))

y_train2 = np.transpose(y_train2,axes=(0,2,1))
y_test2=np.transpose(y_test2,axes=(0,2,1))
y_val2 = np.transpose(y_val2,axes=(0,2,1))

y_train3 = np.transpose(y_train3,axes=(0,2,1))
y_test3=np.transpose(y_test3,axes=(0,2,1))
y_val3 = np.transpose(y_val3,axes=(0,2,1))


print(y_test1.shape, "is post transpose y shape")

# min_lai = np.min(X_train[:,:,8])
# max_lai = np.max(X_train[:,:,8])

# X_train[:,:,8] = (X_train[:,:,8] - min_lai)/ (max_lai - min_lai)
# X_val[:,:,8] = (X_val[:,:,8] - min_lai)/ (max_lai - min_lai)
# X_test[:,:,8] = (X_test[:,:,8] - min_lai)/ (max_lai - min_lai)
X_train = (X_train - mean_value)/std_val
X_test = (X_test - mean_value)/std_val
X_val = (X_val - mean_value)/std_val

mean_value_ndvi = mean_value[7] ## prev: [-3]#  mean_value[-1]
mean_value_lai = mean_value[8]# prev: -2
mean_value_vcf = mean_value[9]
# print(mean_value.shape,"is new shape of mean value")
std_val_ndvi = std_val[7]
std_val_lai = std_val[8]
std_val_vcf = std_val[9]


# print('\n\nndvi mean in x train:',mean_value)
y_train1 = (y_train1 - mean_value_ndvi)/std_val_ndvi
# y_test1 = (y_test1 - mean_value_ndvi)/std_val_ndvi
y_val1 = (y_val1 - mean_value_ndvi)/std_val_ndvi

y_train2 = (y_train2 - mean_value_lai)/std_val_lai
# y_test2 = (y_test2 - mean_value_lai)/std_val_lai
y_val2 = (y_val2 - mean_value_lai)/std_val_lai

y_train3 = (y_train3 - mean_value_vcf)/std_val_vcf
# y_test3 = (y_test3 - mean_value_vcf)/std_val_vcf
y_val3 = (y_val3 - mean_value_vcf)/std_val_vcf
print(y_test1.shape, "is output y shape")
print("Note: shape format is now #samples,  # of timesteps, #features or targets in case of y,\n")

print(X_train.shape)

print("Now min/ax of lai is:", np.min(y_train2.flatten()), np.max(y_train2.flatten()))


## Final Reshaping of data to shape/format: (samples, timesteps, features)

### Drop VCF as target

In [ ]:

#final reshaping for multihead model
# You can stack the y_train and y_test variables as 1-d arrays originally as follows:
y_train = np.dstack((y_train1, y_train2))#, y_train3))  # 3D numpy array (note chat wanted column_stack)
y_val = np.dstack((y_val1, y_val2))#, y_val3))
y_test = np.dstack((y_test1, y_test2))#, y_test3))

print(y_test1.shape)
print(y_test.shape)


Test (graph) viz of variables to ensure theyre still on same scale as each other after standardization process


## Save data as .npy

In [ ]:
np.save("X_train.npy", X_train)
np.save("X_val.npy", X_val)
np.save( "X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_val.npy", y_val)
np.save("y_test.npy", y_test)
np.save("y_train1.npy", y_train1)
np.save("y_train2.npy", y_train2)
np.save("y_test1.npy", y_test1)
np.save("y_test2.npy", y_test2)
np.save("y_val1.npy", y_val1)
np.save("y_val2.npy", y_val2) #note: saving stdardized data so will need ot unstd. to get back to real val range
